In [1]:
import pandas as pd
import requests
import gmaps
from pprint import pprint
import sys
import matplotlib.pyplot as plt
# Import API key
sys.path.append('..')
from settings import config

# Configure gmaps API key
gmaps.configure(api_key=config.g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hobart,AU,-42.88,147.33,46.99,93,75,12.75,light rain
1,1,Snyder,US,32.72,-100.92,55.40,54,1,8.05,clear sky
2,2,Kaitangata,NZ,-46.28,169.85,48.99,87,100,3.83,overcast clouds
3,3,Canico,PT,32.63,-16.85,77.00,69,40,12.75,scattered clouds
4,4,Bluff,NZ,-46.60,168.33,46.00,97,100,7.11,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input('What is the minimum temperature you would like? '))
max_temp = float(input('What is the maximum temperature you would like? '))

What is the minimum temperature you would like? 40
What is the maximum temperature you would like? 60


In [17]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preffered_city = city_data_df.loc[(city_data_df['Max Temp']>=min_temp) & (city_data_df['Max Temp']<=max_temp)]

ValueError: Can only compare identically-labeled Series objects

In [ ]:
# 4a. Determine if there are any empty rows.
preffered_city.isnull().sum()

In [ ]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preffered_city.dropna()

In [ ]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()


In [ ]:
# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""

In [ ]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": config.g_key,
    "location": ""
}

In [18]:
# 6b. Iterate through the hotel DataFrame.
for index,row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params["location"] = f'{lat},{lng}'
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'

# 6e. Make request and retrieve the JSON data from the search. 
    hotel = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_name = hotel['results'][0]['name']
        hotel_df.loc[index,'Hotel Name'] = hotel_name
    except:
        print('Hotel name not found for city: ',row['City'])
        pass

Hotel name not found for city:  Albany
Hotel name not found for city:  Uinskoye
Hotel name not found for city:  Lugovoy
Hotel name not found for city:  Moba
Hotel name not found for city:  Arman
Hotel name not found for city:  Elko
Hotel name not found for city:  Okhotsk
Hotel name not found for city:  Troitskoye
Hotel name not found for city:  Teya
Hotel name not found for city:  Kaz
Hotel name not found for city:  Avera
Hotel name not found for city:  Parfenyevo
Hotel name not found for city:  Vohma
Hotel name not found for city:  Choya
Hotel name not found for city:  Kuldur
Hotel name not found for city:  Svetlaya
Hotel name not found for city:  Saint-Augustin


In [19]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hobart,AU,46.99,light rain,-42.88,147.33,St Ives Hobart Accommodation Tasmania
1,Snyder,US,55.40,clear sky,32.72,-100.92,Holiday Inn Express & Suites Snyder
2,Kaitangata,NZ,48.99,overcast clouds,-46.28,169.85,Kaitangata Motor Camp
4,Bluff,NZ,46.00,overcast clouds,-46.60,168.33,Bluff Homestead - Guesthouse & Campervan Park
10,Kjollefjord,NO,48.76,few clouds,70.95,27.35,Hotel Nordkyn


In [20]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [21]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt><b>Hotel Name</b></dt><dt>{Hotel Name}</dt>
<dt><b>City</b></dt><dt>{City}</dt>
<dt><b>Country Code</b></dt><dt>{Country}</dt>
<dt><b>Weather Description</b></dt><dt>{Current Description}</dt>
<dt><b>Max Temp</b></dt><dt>{Max Temp} °F</dt>
</dl>
"""

In [22]:
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

In [15]:
# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df['Max Temp']
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations,weights=max_temp, max_intensity=300, dissipating =False, point_radius=4 )
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))